# Site selection criteria for a new Peruvian restaurant in Chicago

## Introduction

This project aims to find potential areas to open a new Peruvian restaurant within and around the Central Business District (CBD) in Chicago. In order to find those areas, different variables were extracted or built at that area level. These areas were represented by census tracts from which were extracted 2018 estimated Census variables, 2018-2019 crime events and recent location data for food category from Foursquare. These variables were normalized and sum up to provide a score for census tracts. The higher the score, the more potential is the census tract to open a new Peruvian restaurant.

Let's start downloading all the dependencies that we will need for this project.

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import sys
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests. Requests allows you to send organic, grass-fed HTTP/1.1 requests, without the need for manual labor
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Folium installed')
print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Folium installed
Libraries imported.


## Data Pre-processing

In this section, data was prepared to be ready for analysis and get results. First of all, Census data from United States was donwloaded as csv files and stored in the local machine. Each table corresponds to a Census variable for each census tracts within and around the CBD in Chicago. These tables were: 2018 estimated population, 2018 estimated median income (dollars) of the last 12 months for households, 2018 estimated unemployment rate, and 2018 estimated latino population.

In [2]:
# Reading population table and storing it as a dataframe
pop = pd.read_csv('Population.csv', sep = ';')
print(pop.shape) # Validating number of rows
pop.head() # See first 5 rows

(206, 458)


,GEO_ID,NAME,S0101_C01_001E,S0101_C01_001M,S0101_C02_001E,S0101_C02_001M,S0101_C03_001E,S0101_C03_001M,S0101_C04_001E,S0101_C04_001M,S0101_C05_001E,S0101_C05_001M,S0101_C06_001E,S0101_C06_001M,S0101_C01_002E,S0101_C01_002M,S0101_C02_002E,S0101_C02_002M,S0101_C03_002E,S0101_C03_002M,S0101_C04_002E,S0101_C04_002M,S0101_C05_002E,S0101_C05_002M,S0101_C06_002E,S0101_C06_002M,S0101_C01_003E,S0101_C01_003M,S0101_C02_003E,S0101_C02_003M,S0101_C03_003E,S0101_C03_003M,S0101_C04_003E,S0101_C04_003M,S0101_C05_003E,S0101_C05_003M,S0101_C06_003E,S0101_C06_003M,S0101_C01_004E,S0101_C01_004M,S0101_C02_004E,S0101_C02_004M,S0101_C03_004E,S0101_C03_004M,S0101_C04_004E,S0101_C04_004M,S0101_C05_004E,S0101_C05_004M,S0101_C06_004E,S0101_C06_004M,S0101_C01_005E,S0101_C01_005M,S0101_C02_005E,S0101_C02_005M,S0101_C03_005E,S0101_C03_005M,S0101_C04_005E,S0101_C04_005M,S0101_C05_005E,S0101_C05_005M,S0101_C06_005E,S0101_C06_005M,S0101_C01_006E,S0101_C01_006M,S0101_C02_006E,S0101_C02_006M,S0101_C03_006E,S0101_C03_006M,S0101_C04_006E,S0101_C04_006M,S0101_C05_006E,S0101_C05_006M,S0101_C06_006E,S0101_C06_006M,S0101_C01_007E,S0101_C01_007M,S0101_C02_007E,S0101_C02_007M,S0101_C03_007E,S0101_C03_007M,S0101_C04_007E,S0101_C04_007M,S0101_C05_007E,S0101_C05_007M,S0101_C06_007E,S0101_C06_007M,S0101_C01_008E,S0101_C01_008M,S0101_C02_008E,S0101_C02_008M,S0101_C03_008E,S0101_C03_008M,S0101_C04_008E,S0101_C04_008M,S0101_C05_008E,S0101_C05_008M,S0101_C06_008E,S0101_C06_008M,S0101_C01_009E,S0101_C01_009M,S0101_C02_009E,S0101_C02_009M,S0101_C03_009E,S0101_C03_009M,S0101_C04_009E,S0101_C04_009M,S0101_C05_009E,S0101_C05_009M,S0101_C06_009E,S0101_C06_009M,S0101_C01_010E,S0101_C01_010M,S0101_C02_010E,S0101_C02_010M,S0101_C03_010E,S0101_C03_010M,S0101_C04_010E,S0101_C04_010M,S0101_C05_010E,S0101_C05_010M,S0101_C06_010E,S0101_C06_010M,S0101_C01_011E,S0101_C01_011M,S0101_C02_011E,S0101_C02_011M,S0101_C03_011E,S0101_C03_011M,S0101_C04_011E,S0101_C04_011M,S0101_C05_011E,S0101_C05_011M,S0101_C06_011E,S0101_C06_011M,S0101_C01_012E,S0101_C01_012M,S0101_C02_012E,S0101_C02_012M,S0101_C03_012E,S0101_C03_012M,S0101_C04_012E,S0101_C04_012M,S0101_C05_012E,S0101_C05_012M,S0101_C06_012E,S0101_C06_012M,S0101_C01_013E,S0101_C01_013M,S0101_C02_013E,S0101_C02_013M,S0101_C03_013E,S0101_C03_013M,S0101_C04_013E,S0101_C04_013M,S0101_C05_013E,S0101_C06_013E,S0101_C06_013M,S0101_C01_014E,S0101_C01_014M,S0101_C02_014E,S0101_C02_014M,S0101_C03_014E,S0101_C03_014M,S0101_C04_014E,S0101_C04_014M,S0101_C05_014E,S0101_C05_014M,S0101_C05_013M,S0101_C06_014E,S0101_C06_014M,S0101_C01_015E,S0101_C01_015M,S0101_C02_015E,S0101_C02_015M,S0101_C03_015E,S0101_C03_015M,S0101_C04_015E,S0101_C04_015M,S0101_C05_015E,S0101_C05_015M,S0101_C06_015E,S0101_C06_015M,S0101_C01_016E,S0101_C01_016M,S0101_C02_016E,S0101_C02_016M,S0101_C03_016E,S0101_C03_016M,S0101_C04_016E,S0101_C04_016M,S0101_C05_016E,S0101_C05_016M,S0101_C06_016E,S0101_C06_016M,S0101_C01_017E,S0101_C01_017M,S0101_C02_017E,S0101_C02_017M,S0101_C03_017E,S0101_C03_017M,S0101_C04_017E,S0101_C04_017M,S0101_C05_017E,S0101_C05_017M,S0101_C06_017E,S0101_C06_017M,S0101_C01_018E,S0101_C01_018M,S0101_C02_018E,S0101_C02_018M,S0101_C03_018E,S0101_C03_018M,S0101_C04_018E,S0101_C04_018M,S0101_C05_018E,S0101_C05_018M,S0101_C06_018E,S0101_C06_018M,S0101_C01_019E,S0101_C01_019M,S0101_C02_019E,S0101_C02_019M,S0101_C03_019E,S0101_C03_019M,S0101_C04_019E,S0101_C04_019M,S0101_C05_019E,S0101_C05_019M,S0101_C06_019E,S0101_C06_019M,S0101_C01_020E,S0101_C01_020M,S0101_C02_020E,S0101_C02_020M,S0101_C03_020E,S0101_C03_020M,S0101_C04_020E,S0101_C04_020M,S0101_C05_020E,S0101_C05_020M,S0101_C06_020E,S0101_C06_020M,S0101_C01_021E,S0101_C01_021M,S0101_C02_021E,S0101_C02_021M,S0101_C03_021E,S0101_C03_021M,S0101_C04_021E,S0101_C04_021M,S0101_C05_021E,S0101_C05_021M,S0101_C06_021E,S0101_C06_021M,S0101_C01_022E,S0101_C01_022M,S0101_C02_022E,S0101_C02_022M,S0101_C03_022E,S0101_C03_022M,S0101_C04_022E,S0101_C04_022M,S0101_C05_022E,S0101_C05_022M,S0101_C06_022E,S0101_C06_022M,S0101_C01_023E,S0101_C01_02

As it can be seen, there are a lot of variables for this population table. However, only 2 variables are needed. They are: 'GEO_ID' which is the key variable for census tracts and 'S0101_C01_001E' which is the 2018 estimated total population.

In [3]:
# Creating a new dataframe
pop = pop[['GEO_ID', 'S0101_C01_001E']]
pop.rename(columns={'S0101_C01_001E':'population'}, inplace = True) # Renaming column for appropiate reading
print(pop.shape)
pop.head()

(206, 2)


,GEO_ID,population
0,1400000US17031051300,2826
1,1400000US17031051400,1796
2,1400000US17031060900,7114
3,1400000US17031061500,1888
4,1400000US17031061800,1340


Now, reading the income table and storing it as a dataframe with only 2 columns needed.

In [4]:
income = pd.read_csv('Income.csv', sep = ';')
income = income[['GEO_ID', 'S1901_C01_012E']] # S1901_C01_012E is the household median income
income.rename(columns={'S1901_C01_012E':'income'}, inplace = True) # Renaming column for appropiate reading
print(income.shape)
income.head()

(206, 2)


,GEO_ID,income
0,1400000US17031051300,178750
1,1400000US17031320400,176000
2,1400000US17031062700,168352
3,1400000US17031071700,162054
4,1400000US17031831000,159020


Reading unemployment table.

In [5]:
unemployment = pd.read_csv('Unemployment.csv', sep = ';')
unemployment = unemployment[['GEO_ID', 'S2301_C04_001E']] # S2301_C04_001E is the unemployment rate
unemployment.rename(columns={'S2301_C04_001E':'unemployment'}, inplace = True) # Renaming column for appropiate reading
print(unemployment.shape)
unemployment.head()

(206, 2)


,GEO_ID,unemployment
0,1400000US17031051300,1.3
1,1400000US17031051400,3.3
2,1400000US17031060900,1.9
3,1400000US17031061500,2.7
4,1400000US17031061800,4.6


Finally, reading the last Census variable: Latinos.

In [6]:
latinos = pd.read_csv('Latinos.csv', sep = ';')
latinos = latinos[['GEO_ID', 'B03001_003E']] # B03001_003E is the latino total population
latinos.rename(columns={'B03001_003E':'latinos'}, inplace = True) # Renaming column for appropiate reading
print(latinos.shape)
latinos.head()

(206, 2)


,GEO_ID,latinos
0,1400000US17031071700,70
1,1400000US17031070700,527
2,1400000US17031280100,287
3,1400000US17031350100,83
4,1400000US17031230900,2818


Additionally, density population variable is needed. In that sense, areas in hectareas were obtained from a shapefile of census tracts (which then was exported as a json file and used for mapping later in the section exploratory data analysis). The area obtained from the shapefile was exported as a csv file.

In [7]:
areas = pd.read_csv('AreaCTracts.csv', sep = ';')
areas.rename(columns={'AFFGEOID':'GEO_ID', 'ALANDHA':'area'}, inplace = True) # Renaming columns for appropiate reading
print(areas.shape)
areas.head()

(206, 2)


,GEO_ID,area
0,1400000US17031243000,32.4548
1,1400000US17031301100,21.1425
2,1400000US17031351100,34.7265
3,1400000US17031837300,36.8498
4,1400000US17031839500,26.2139


Another variable of interest was the crime rate. This data was downloaded as point data and contained crimes within or out of restaurants in next categories: assault, battery, burglary, robbery and theft. Once donwloaded was georeferenced in a GIS program and imputed the key variable of census tracts (GEO_ID). Then, crimes were exported as a csv file.

In [8]:
crime = pd.read_csv('Crime_Restaurants.csv')
crime.rename(columns={'AFFGEOID':'GEO_ID'}, inplace = True)
crime.head()

,FID_1,Case_Numbe,Date,Block,IUCR,Primary_Ty,Descriptio,Location_D,Arrest,Domestic,Beat,Ward,FBI_Code,X_Coordina,Y_Coordina,Year,Latitude,Longitude,Location,GEO_ID
0,1560,JB242135,04/29/2018 02:03:00 AM,007XX W GARFIELD BLVD,460,BATTERY,SIMPLE,RESTAURANT,False,False,935.0,3.0,08B,1172268.0,1868397.0,2018.0,41.794326,-87.643832,"(41.79432621, -87.643832227)",1400000US17031843800
1,1887,JB273427,05/22/2018 08:45:00 AM,007XX W 54TH ST,560,ASSAULT,SIMPLE,RESTAURANT,False,False,935.0,3.0,08A,1172336.0,1869065.0,2018.0,41.796158,-87.643563,"(41.796157777, -87.64356322)",1400000US17031843800
2,35,JB101408,43132.411111,020XX W CHICAGO AVE,820,THEFT,$500 AND UNDER,RESTAURANT,False,False,1221.0,1.0,6,1162746.0,1905337.0,2018.0,41.895898,-87.677715,"(41.895897635, -87.677715036)",1400000US17031243000
3,118,JB109958,43344.416667,020XX W CHICAGO AVE,820,THEFT,$500 AND UNDER,RESTAURANT,False,False,1221.0,1.0,6,1162746.0,1905337.0,2018.0,41.895898,-87.677715,"(41.895897635, -87.677715036)",1400000US17031243000
4,911,JB184609,03/13/2018 05:00:00 PM,020XX W CHICAGO AVE,560,ASSAULT,SIMPLE,RESTAURANT,False,False,1221.0,1.0,08A,1162746.0,1905337.0,2018.0,41.895898,-87.677715,"(41.895897635, -87.677715036)",1400000US17031243000


Since our unit analysis is the census tract, crimes were aggregated to that level to obtain the number of crimes by census tract.

In [9]:
df_crime = crime.groupby('GEO_ID').count().reset_index()
df_crime = df_crime[['GEO_ID', 'FID_1']]
df_crime.rename(columns={'FID_1':'crime count'}, inplace = True)
df_crime.head()

,GEO_ID,crime count
0,1400000US17031010100,3
1,1400000US17031010201,14
2,1400000US17031010202,21
3,1400000US17031010300,1
4,1400000US17031010400,6


Then, all the previous tables were merged by using the key variable 'GEO_ID' in a new dataframe called 'df'.

In [10]:
df = pop
df = pd.merge(df, income, on = 'GEO_ID', how = 'left')
df = pd.merge(df, unemployment, on = 'GEO_ID', how = 'left')
df = pd.merge(df, latinos, on = 'GEO_ID', how = 'left')
df = pd.merge(df, areas, on = 'GEO_ID', how = 'left')
df = pd.merge(df, df_crime, on = 'GEO_ID', how = 'left')
df

,GEO_ID,population,income,unemployment,latinos,area,crime count
0,1400000US17031051300,2826,178750,1.3,185,32.2017,1.0
1,1400000US17031051400,1796,116364,3.3,143,34.6283,NaN
2,1400000US17031060900,7114,59942,1.9,1162,49.9067,4.0
3,1400000US17031061500,1888,125000,2.7,175,16.2913,1.0
4,1400000US17031061800,1340,73523,4.6,142,8.2769,3.0
5,1400000US17031061901,3597,60909,3.3,289,16.1112,16.0
6,1400000US17031061902,4653,64508,7.8,139,27.5608,NaN
7,1400000US17031062000,2981,80595,2.2,154,22.4975,6.0
8,1400000US17031062100,3772,94500,4.5,315,24.4805,36.0
9,1400000US17031062200,2719,119392,2.3,309,24.2322,5.0


Let's check our variables are the right type for analysis.

In [11]:
df.dtypes

GEO_ID           object
population        int64
income            int64
unemployment    float64
latinos           int64
area            float64
crime count     float64
dtype: object

Income must be a float type and crime count as integer. Before that, let's clean our data. When working with census tables there are some weird expressions like '-' and '**' which must be replaced by 'NaN' to make sure that we can do the correct analysis on our variables. Then, replace NaN by 0 on the crime count variables (No NaN were visualized in the remaining variables). Drop NaN cells in our dataframe.

In [12]:
df.replace("-", np.nan, inplace = True)
df.replace("**", np.nan, inplace = True)
df['crime count'].replace(np.nan, 0, inplace=True)
df.dropna(inplace=True)
df.reset_index(drop = True, inplace = True)
print(df.shape)

(206, 7)


Let's modify the type of data in the variables mentioned before.

In [13]:
df[["income"]] = df[["income"]].astype("float")
df[["crime count"]] = df[["crime count"]].astype("int64")
df.dtypes

GEO_ID           object
population        int64
income          float64
unemployment    float64
latinos           int64
area            float64
crime count       int64
dtype: object

Let's check for nulls

In [14]:
df.isnull().sum()

GEO_ID          0
population      0
income          0
unemployment    0
latinos         0
area            0
crime count     0
dtype: int64

No nulls are present in our data and all variables have the correct data types. Now, new variables need to be build to represent our variables of analysis. First, by having population and area, we can calculate the density variable. Next, let's calculate the latino population rate. Then, let's build the crime rate inflated by 1,000 persons to avoid very short numbers. Finally, let's represent the income by thousand of dollars for appropiate reading for mapping purposes.

In [15]:
df["density"] = df["population"]/df["area"]
df["latinos rate"] = (df["latinos"]*100)/df["population"]
df["crime rate"] = (df["crime count"]*1000)/df["population"]
df["income"] = df["income"]/1000 # in thousands of dollars
df.head()

,GEO_ID,population,income,unemployment,latinos,area,crime count,density,latinos rate,crime rate
0,1400000US17031051300,2826,178.750,1.3,185,32.2017,1,87.759342,6.546355,0.353857
1,1400000US17031051400,1796,116.364,3.3,143,34.6283,0,51.865093,7.962138,0.000000
2,1400000US17031060900,7114,59.942,1.9,1162,49.9067,4,142.545991,16.333989,0.562272
3,1400000US17031061500,1888,125.000,2.7,175,16.2913,1,115.890076,9.269068,0.529661
4,1400000US17031061800,1340,73.523,4.6,142,8.2769,3,161.896362,10.597015,2.238806


Our variables of analysis until now are: income, unemployment, density, latinos rate and crime rate. On the other hand, our last built variables comes from 'Foursquare'. A first variable is related to the number of restaurants in each census tracts where the higher the number, the lesser the chance to consider a census tract as potential for a new restaurant. And the second variable is related to the top 2 most common restaurants. If one of them is a Latin American restaurant, the census tract will be considered less potential. To obtain these variables, coordinates of census tracts centroids in Chicago were needed and downloaded from the USA Census. Then, the coordinates table was cleaned and merged to our dataframe of analysis 'df' to start finding food venues using the Foursquare API. From there, the 2 variables needed were built from 'Foursquare'.

In [16]:
# Reading geographical centroids of census tracts
link = 'https://www2.census.gov/geo/docs/reference/cenpop2010/tract/CenPop2010_Mean_TR17.txt'
df_coordinates = pd.read_csv(link)
print(df_coordinates.shape) # Validating number of rows
df_coordinates.head() # It contains all census tracts within the county of Illinois, but we will need some of them

(3123, 6)


,STATEFP,COUNTYFP,TRACTCE,POPULATION,LATITUDE,LONGITUDE
0,17,1,100,4627,39.943809,-91.362724
1,17,1,201,1986,39.953308,-91.387569
2,17,1,202,2999,39.941252,-91.386019
3,17,1,400,4322,39.948390,-91.406128
4,17,1,500,2337,39.943484,-91.398623


In [17]:
# The first 3 variables must be strings
df_coordinates.dtypes

STATEFP         int64
COUNTYFP        int64
TRACTCE         int64
POPULATION      int64
LATITUDE      float64
LONGITUDE     float64
dtype: object

In [18]:
# Cleaning data
df_coordinates = df_coordinates[df_coordinates['COUNTYFP'] == 31] # Filtering census tracts in Cook county where Chicago belongs
df_coordinates.drop(['POPULATION'], axis = 1, inplace = True) # Don't need population variable as we have it
df_coordinates.reset_index(drop = True, inplace = True) # Reseting index
df_coordinates[["STATEFP"]] = df_coordinates[["STATEFP"]].astype("str") # Converting STATEFP to string
df_coordinates[["COUNTYFP"]] = df_coordinates[["COUNTYFP"]].astype("str") # Converting COUNTYFP to string
df_coordinates[["TRACTCE"]] = df_coordinates[["TRACTCE"]].astype("str") # Converting TRACTCE to string
# COUNTYFP is a 3 digits variable, some we need to add a zero
df_coordinates["COUNTYFP"] = "0" + df_coordinates["COUNTYFP"]
df_coordinates.head() # Let's see how our coordinates data looks

,STATEFP,COUNTYFP,TRACTCE,LATITUDE,LONGITUDE
0,17,031,10100,42.021200,-87.669534
1,17,031,10201,42.015972,-87.680738
2,17,031,10202,42.015540,-87.672469
3,17,031,10300,42.016013,-87.666414
4,17,031,10400,42.004091,-87.659582


In [19]:
# We need a key variable similar to the GEO_ID in our dataframe
# To have that similar variable we need a series of steps as follows
df_coordinates['len_TRACTCE'] = df_coordinates['TRACTCE'].apply(len) # Calculating length of TRACTCE
df_coordinates.loc[df_coordinates['len_TRACTCE'] == 5, 'TRACT'] = "0" + df_coordinates["TRACTCE"] # If the length is 5, let's add a zero because TRACTCE is a 6 digits variable
df_coordinates.loc[df_coordinates['len_TRACTCE'] == 6, 'TRACT'] = df_coordinates["TRACTCE"] # If the length is 6, keep the variable as it is
df_coordinates["GEO_ID"] = "1400000US" + df_coordinates["STATEFP"] + df_coordinates["COUNTYFP"] + df_coordinates["TRACT"] # Concatenates 1400000US + STATEFP + COUNTYFP + TRACT in a new variable called GEO_ID
df_coordinates.drop(['STATEFP', 'COUNTYFP', 'TRACTCE', 'TRACT', 'len_TRACTCE'], axis = 1, inplace = True) # And drop variables we won't need
df_coordinates.rename(columns={'LATITUDE':'Latitude', 'LONGITUDE':'Longitude'}, inplace = True) #Renaming variables
print(df_coordinates.shape)
df_coordinates.head()

(1319, 3)


,Latitude,Longitude,GEO_ID
0,42.021200,-87.669534,1400000US17031010100
1,42.015972,-87.680738,1400000US17031010201
2,42.015540,-87.672469,1400000US17031010202
3,42.016013,-87.666414,1400000US17031010300
4,42.004091,-87.659582,1400000US17031010400


As it can be seen, we have the coordinates of census tracts centroids with a key variable 'GEO_ID' and is similar to our key variable in our main dataframe, hence the match between both dataframes will be successful. Let's check null in any case.

In [20]:
df_coordinates.isnull().sum()

Latitude     0
Longitude    0
GEO_ID       0
dtype: int64

Merging the two dataframes keeping all rows of our main dataframe with census and crime variable (left)

In [21]:
df_merged = pd.merge(df, df_coordinates, on = 'GEO_ID', how = 'left')
df_merged

,GEO_ID,population,income,unemployment,latinos,area,crime count,density,latinos rate,crime rate,Latitude,Longitude
0,1400000US17031051300,2826,178.750,1.3,185,32.2017,1,87.759342,6.546355,0.353857,41.935755,-87.676010
1,1400000US17031051400,1796,116.364,3.3,143,34.6283,0,51.865093,7.962138,0.000000,41.936528,-87.680827
2,1400000US17031060900,7114,59.942,1.9,1162,49.9067,4,142.545991,16.333989,0.562272,41.949074,-87.645666
3,1400000US17031061500,1888,125.000,2.7,175,16.2913,1,115.890076,9.269068,0.529661,41.945462,-87.661520
4,1400000US17031061800,1340,73.523,4.6,142,8.2769,3,161.896362,10.597015,2.238806,41.946435,-87.646811
5,1400000US17031061901,3597,60.909,3.3,289,16.1112,16,223.260837,8.034473,4.448151,41.942988,-87.643348
6,1400000US17031061902,4653,64.508,7.8,139,27.5608,0,168.826739,2.987320,0.000000,41.942590,-87.641151
7,1400000US17031062000,2981,80.595,2.2,154,22.4975,6,132.503612,5.166052,2.012747,41.942830,-87.647034
8,1400000US17031062100,3772,94.500,4.5,315,24.4805,36,154.081820,8.351007,9.544008,41.942945,-87.651583
9,1400000US17031062200,2719,119.392,2.3,309,24.2322,5,112.206073,11.364472,1.838911,41.942525,-87.656674


Now, we are going to start utilizing the Foursquare API to explore food venues

In [22]:
CLIENT_ID = 'your id here' # Foursquare ID
CLIENT_SECRET = 'your id here' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [23]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Let's create a function to get food venues in all census tracts in Chicago defining a radius of 300m. for each census tract and a limit of 100 venues returned by Foursquare API.

In [24]:
LIMIT = 100

def getNearbyRestaurants(names, latitudes, longitudes, radius=300, categoryId=''):
    restaurant_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
                    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        if(categoryId!=''):
            url=url+'&categoryId={}'
            url=url.format(categoryId)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # retrun only the venue name, corrdinates and catagoery
        restaurant_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_restaurant = pd.DataFrame([item for restaurant_list in restaurant_list for item in restaurant_list])
    nearby_restaurant.columns = ['GEO_ID', 
                  'GEO_ID Latitude', 
                  'GEO_ID Longitude', 
                  'Restaurant', 
                  'Restaurant Latitude', 
                  'Restaurant Longitude', 
                  'Restaurant Category']
    
    return(nearby_restaurant)   

Let's use the function on our merged and main dataframe using the 'category id' for food venues: 4d4b7105d754a06374d81259 and create a new dataframe called 'all_restaurants'.

In [25]:
all_restaurants = getNearbyRestaurants(names = df_merged['GEO_ID'],
                                       latitudes = df_merged['Latitude'], 
                                       longitudes = df_merged['Longitude'], 
                                       radius = 300, 
                                       categoryId = '4d4b7105d754a06374d81259')

Let's check how many food venues we got.

In [26]:
print(all_restaurants.shape) # we have 2,320 food venues in our dataframe
all_restaurants.head()

(2320, 7)


,GEO_ID,GEO_ID Latitude,GEO_ID Longitude,Restaurant,Restaurant Latitude,Restaurant Longitude,Restaurant Category
0,1400000US17031051300,41.935755,-87.676010,Desch's BBQ Shack,41.937097,-87.674744,BBQ Joint
1,1400000US17031051300,41.935755,-87.676010,Redhot Ranch,41.937634,-87.673618,Hot Dog Joint
2,1400000US17031051400,41.936528,-87.680827,Herbivore Heaven,41.934212,-87.679794,Vegetarian / Vegan Restaurant
3,1400000US17031060900,41.949074,-87.645666,Angelina Ristorante,41.947218,-87.647108,Italian Restaurant
4,1400000US17031060900,41.949074,-87.645666,Panino's,41.949406,-87.649014,Pizza Place


Let's check all the categories we got.

In [27]:
print('There are {} uniques categories.'.format(len(all_restaurants['Restaurant Category'].unique())))

There are 95 uniques categories.


And explore them.

In [28]:
all_restaurants.groupby('Restaurant Category').count()

,GEO_ID,GEO_ID Latitude,GEO_ID Longitude,Restaurant,Restaurant Latitude,Restaurant Longitude
Restaurant Category,,,,,,
African Restaurant,1,1,1,1,1,1
American Restaurant,125,125,125,125,125,125
Argentinian Restaurant,2,2,2,2,2,2
Asian Restaurant,42,42,42,42,42,42
BBQ Joint,31,31,31,31,31,31
Bagel Shop,14,14,14,14,14,14
Bakery,72,72,72,72,72,72
Bistro,4,4,4,4,4,4
Brazilian Restaurant,6,6,6,6,6,6


Since, we are competing with restaurants, we need to filter those restaurants on this dataframe. So, let's create a new dataframe filtering rows that contain 'restaurants'.

In [29]:
chicago_venues = all_restaurants[all_restaurants['Restaurant Category'].str.contains("Restaurant")]
chicago_venues.reset_index(drop = True, inplace = True) # Reseting index of rows
print(chicago_venues.shape) # we have 1,286 food venues in our dataframe
chicago_venues.head()

(1286, 7)


,GEO_ID,GEO_ID Latitude,GEO_ID Longitude,Restaurant,Restaurant Latitude,Restaurant Longitude,Restaurant Category
0,1400000US17031051400,41.936528,-87.680827,Herbivore Heaven,41.934212,-87.679794,Vegetarian / Vegan Restaurant
1,1400000US17031060900,41.949074,-87.645666,Angelina Ristorante,41.947218,-87.647108,Italian Restaurant
2,1400000US17031060900,41.949074,-87.645666,Jai Yen,41.950133,-87.648981,Japanese Restaurant
3,1400000US17031061500,41.945462,-87.661520,Ella Elli,41.945208,-87.663652,American Restaurant
4,1400000US17031061500,41.945462,-87.661520,Crosby's Kitchen,41.945185,-87.663830,American Restaurant


Now. Let's create a new dataframe with the number of restaurants for each census tract. This will be our 6th variable of analysis.

In [30]:
number_restaurants = chicago_venues.groupby('GEO_ID').count().reset_index()
number_restaurants = number_restaurants[['GEO_ID', 'GEO_ID Latitude']]
number_restaurants.rename(columns={'GEO_ID Latitude':'number restaurants'}, inplace = True)
number_restaurants

,GEO_ID,number restaurants
0,1400000US17031051400,1
1,1400000US17031060900,2
2,1400000US17031061500,4
3,1400000US17031061800,9
4,1400000US17031061901,10
5,1400000US17031061902,5
6,1400000US17031062000,16
7,1400000US17031062100,22
8,1400000US17031062200,8
9,1400000US17031062300,5


And merge this new dataframe to our main dataframe in a new variable 'df_merged2'.

In [31]:
df_merged2 = pd.merge(df_merged, number_restaurants, on = 'GEO_ID', how = 'left')
df_merged2

,GEO_ID,population,income,unemployment,latinos,area,crime count,density,latinos rate,crime rate,Latitude,Longitude,number restaurants
0,1400000US17031051300,2826,178.750,1.3,185,32.2017,1,87.759342,6.546355,0.353857,41.935755,-87.676010,NaN
1,1400000US17031051400,1796,116.364,3.3,143,34.6283,0,51.865093,7.962138,0.000000,41.936528,-87.680827,1.0
2,1400000US17031060900,7114,59.942,1.9,1162,49.9067,4,142.545991,16.333989,0.562272,41.949074,-87.645666,2.0
3,1400000US17031061500,1888,125.000,2.7,175,16.2913,1,115.890076,9.269068,0.529661,41.945462,-87.661520,4.0
4,1400000US17031061800,1340,73.523,4.6,142,8.2769,3,161.896362,10.597015,2.238806,41.946435,-87.646811,9.0
5,1400000US17031061901,3597,60.909,3.3,289,16.1112,16,223.260837,8.034473,4.448151,41.942988,-87.643348,10.0
6,1400000US17031061902,4653,64.508,7.8,139,27.5608,0,168.826739,2.987320,0.000000,41.942590,-87.641151,5.0
7,1400000US17031062000,2981,80.595,2.2,154,22.4975,6,132.503612,5.166052,2.012747,41.942830,-87.647034,16.0
8,1400000US17031062100,3772,94.500,4.5,315,24.4805,36,154.081820,8.351007,9.544008,41.942945,-87.651583,22.0
9,1400000US17031062200,2719,119.392,2.3,309,24.2322,5,112.206073,11.364472,1.838911,41.942525,-87.656674,8.0


Since we don't have restaurants in some census tracts, we need to fill them with zero.

In [32]:
df_merged2['number restaurants'].replace(np.nan, 0, inplace=True)
df_merged2

,GEO_ID,population,income,unemployment,latinos,area,crime count,density,latinos rate,crime rate,Latitude,Longitude,number restaurants
0,1400000US17031051300,2826,178.750,1.3,185,32.2017,1,87.759342,6.546355,0.353857,41.935755,-87.676010,0.0
1,1400000US17031051400,1796,116.364,3.3,143,34.6283,0,51.865093,7.962138,0.000000,41.936528,-87.680827,1.0
2,1400000US17031060900,7114,59.942,1.9,1162,49.9067,4,142.545991,16.333989,0.562272,41.949074,-87.645666,2.0
3,1400000US17031061500,1888,125.000,2.7,175,16.2913,1,115.890076,9.269068,0.529661,41.945462,-87.661520,4.0
4,1400000US17031061800,1340,73.523,4.6,142,8.2769,3,161.896362,10.597015,2.238806,41.946435,-87.646811,9.0
5,1400000US17031061901,3597,60.909,3.3,289,16.1112,16,223.260837,8.034473,4.448151,41.942988,-87.643348,10.0
6,1400000US17031061902,4653,64.508,7.8,139,27.5608,0,168.826739,2.987320,0.000000,41.942590,-87.641151,5.0
7,1400000US17031062000,2981,80.595,2.2,154,22.4975,6,132.503612,5.166052,2.012747,41.942830,-87.647034,16.0
8,1400000US17031062100,3772,94.500,4.5,315,24.4805,36,154.081820,8.351007,9.544008,41.942945,-87.651583,22.0
9,1400000US17031062200,2719,119.392,2.3,309,24.2322,5,112.206073,11.364472,1.838911,41.942525,-87.656674,8.0


So now we have 6 variables of analysis to determine where are the potential census tracts to open a new Peruvian restaurant. Now we need 2 additional variables that show the top 2 most common restaurants in each census tracts. If one of them is a Latin American restaurant, the chances of opening a new restaurants are lower. Let's identify if we have latin american restaurants.

In [33]:
chicago_venues.groupby('Restaurant Category').count().reset_index()

,Restaurant Category,GEO_ID,GEO_ID Latitude,GEO_ID Longitude,Restaurant,Restaurant Latitude,Restaurant Longitude
0,African Restaurant,1,1,1,1,1,1
1,American Restaurant,125,125,125,125,125,125
2,Argentinian Restaurant,2,2,2,2,2,2
3,Asian Restaurant,42,42,42,42,42,42
4,Brazilian Restaurant,6,6,6,6,6,6
5,Cajun / Creole Restaurant,5,5,5,5,5,5
6,Caribbean Restaurant,10,10,10,10,10,10
7,Chinese Restaurant,68,68,68,68,68,68
8,Comfort Food Restaurant,3,3,3,3,3,3
9,Cuban Restaurant,11,11,11,11,11,11


From this list, we might consider Latin American restaurants those rows with the following nations: Argentinian, Brazilian, Caribbean, Cuban, Latin American, Mexican, Peruvian and South American. These characteristics will help us to define scores for modeling later. Now, let's identify the top 2 most common restaurants in each census tracts following the next process:

In [34]:
# One hot encoding
chicago_onehot = pd.get_dummies(chicago_venues[['Restaurant Category']], prefix = "", prefix_sep = "")

# Add neighborhood column back to dataframe
chicago_onehot['GEO_ID'] = chicago_venues['GEO_ID'] 

# Move neighborhood column to the first column
fixed_columns = [chicago_onehot.columns[-1]] + list(chicago_onehot.columns[:-1])
chicago_onehot = chicago_onehot[fixed_columns]

chicago_onehot.head()

,GEO_ID,African Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant,Cuban Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,English Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Greek Restaurant,Halal Restaurant,Hawaiian Restaurant,Hotpot Restaurant,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,North Indian Restaurant,Persian Restaurant,Peruvian Restaurant,Polish Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Scandinavian Restaurant,Seafood Restaurant,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Ukrainian Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,1400000US17031051400,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,1400000US17031060900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1400000US17031060900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1400000US17031061500,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1400000US17031061500,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [35]:
# let's examine the new dataframe size
chicago_onehot.shape

(1286, 63)

Let's group rows by key variable 'GEO_ID' of census tracts and by taking the mean of the frequency of occurrence of each category

In [36]:
chicago_grouped = chicago_onehot.groupby('GEO_ID').mean().reset_index()
chicago_grouped.head()

,GEO_ID,African Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant,Cuban Restaurant,Dim Sum Restaurant,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,English Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Greek Restaurant,Halal Restaurant,Hawaiian Restaurant,Hotpot Restaurant,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,North Indian Restaurant,Persian Restaurant,Peruvian Restaurant,Polish Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Scandinavian Restaurant,Seafood Restaurant,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Ukrainian Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,1400000US17031051400,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1400000US17031060900,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500000,0.5,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1400000US17031061500,0.0,0.75,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.250000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1400000US17031061800,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.111111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.111111,0.0,0.111111,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.222222,0.0,0.0,0.0,0.0,0.0,0.0,0.111111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1400000US17031061901,0.0,0.00,0.0,0.1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.1,0.000000,0.0,0.0,0.0,0.200000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.1,0.0,0.0,0.0,0.1,0.0,0.400000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Let's print each GEO_ID census tracts along with the top 2 most common restaurants

In [37]:
num_top_venues = 2

for hood in chicago_grouped['GEO_ID']:
    print("----"+hood+"----")
    temp = chicago_grouped[chicago_grouped['GEO_ID'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----1400000US17031051400----
                           venue  freq
0  Vegetarian / Vegan Restaurant   1.0
1             African Restaurant   0.0


----1400000US17031060900----
                 venue  freq
0  Japanese Restaurant   0.5
1   Italian Restaurant   0.5


----1400000US17031061500----
                 venue  freq
0  American Restaurant  0.75
1     Sushi Restaurant  0.25


----1400000US17031061800----
                venue  freq
0  Mexican Restaurant  0.33
1          Restaurant  0.22


----1400000US17031061901----
                venue  freq
0    Sushi Restaurant   0.4
1  Mexican Restaurant   0.2


----1400000US17031061902----
              venue  freq
0  Sushi Restaurant   0.6
1  Asian Restaurant   0.2


----1400000US17031062000----
                venue  freq
0  Mexican Restaurant  0.25
1    Sushi Restaurant  0.25


----1400000US17031062100----
                     venue  freq
0       Mexican Restaurant  0.18
1  New American Restaurant  0.09


----1400000US17031062200----
   

                  venue  freq
0  Fast Food Restaurant   0.5
1            Restaurant   0.5


----1400000US17031240900----
                       venue  freq
0  Latin American Restaurant  0.25
1    New American Restaurant  0.25


----1400000US17031241000----
                       venue  freq
0  Latin American Restaurant  0.67
1        American Restaurant  0.33


----1400000US17031241100----
                  venue  freq
0  Caribbean Restaurant   0.5
1    Mexican Restaurant   0.5


----1400000US17031241200----
                venue  freq
0  Mexican Restaurant   0.5
1    Sushi Restaurant   0.5


----1400000US17031241400----
                  venue  freq
0  Fast Food Restaurant  0.22
1      Ramen Restaurant  0.22


----1400000US17031241500----
                venue  freq
0  Italian Restaurant  0.15
1  Mexican Restaurant  0.15


----1400000US17031241600----
                   venue  freq
0  Vietnamese Restaurant  0.25
1       Asian Restaurant  0.25


----1400000US17031242000----
           

               venue  freq
0   Sushi Restaurant  0.50
1  Indian Restaurant  0.25


----1400000US17031833000----
                     venue  freq
0       Italian Restaurant  0.18
1  New American Restaurant  0.12


----1400000US17031833100----
                 venue  freq
0      Thai Restaurant  0.25
1  American Restaurant  0.12


----1400000US17031833300----
                venue  freq
0  Italian Restaurant  0.23
1  Chinese Restaurant  0.15


----1400000US17031836600----
                  venue  freq
0  Fast Food Restaurant   1.0
1    African Restaurant   0.0


----1400000US17031836700----
                venue  freq
0  Chinese Restaurant   1.0
1  African Restaurant   0.0


----1400000US17031836900----
                 venue  freq
0  American Restaurant   1.0
1   African Restaurant   0.0


----1400000US17031837000----
                 venue  freq
0  American Restaurant   1.0
1   African Restaurant   0.0


----1400000US17031837100----
                             venue  freq
0  Southern 

Let's put that into a pandas dataframe. First, let's write a function to sort the venues in descending order.

In [38]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 2 venues for each census tract

In [39]:
num_top_venues = 2

indicators = ['st', 'nd', 'rd']

# Create columns according to number of top venues
columns = ['GEO_ID']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# Create a new dataframe
geoid_venues_sorted = pd.DataFrame(columns = columns)
geoid_venues_sorted['GEO_ID'] = chicago_grouped['GEO_ID']

for ind in np.arange(chicago_grouped.shape[0]):
    geoid_venues_sorted.iloc[ind, 1:] = return_most_common_venues(chicago_grouped.iloc[ind, :], num_top_venues)

geoid_venues_sorted.head()

,GEO_ID,1st Most Common Venue,2nd Most Common Venue
0,1400000US17031051400,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
1,1400000US17031060900,Japanese Restaurant,Italian Restaurant
2,1400000US17031061500,American Restaurant,Sushi Restaurant
3,1400000US17031061800,Mexican Restaurant,Restaurant
4,1400000US17031061901,Sushi Restaurant,Mexican Restaurant


Let's join this dataframe to our main dataframe.

In [40]:
df_merged3 = pd.merge(df_merged2, geoid_venues_sorted, on = 'GEO_ID', how = 'left')
df_merged3.head()

,GEO_ID,population,income,unemployment,latinos,area,crime count,density,latinos rate,crime rate,Latitude,Longitude,number restaurants,1st Most Common Venue,2nd Most Common Venue
0,1400000US17031051300,2826,178.750,1.3,185,32.2017,1,87.759342,6.546355,0.353857,41.935755,-87.676010,0.0,NaN,NaN
1,1400000US17031051400,1796,116.364,3.3,143,34.6283,0,51.865093,7.962138,0.000000,41.936528,-87.680827,1.0,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
2,1400000US17031060900,7114,59.942,1.9,1162,49.9067,4,142.545991,16.333989,0.562272,41.949074,-87.645666,2.0,Japanese Restaurant,Italian Restaurant
3,1400000US17031061500,1888,125.000,2.7,175,16.2913,1,115.890076,9.269068,0.529661,41.945462,-87.661520,4.0,American Restaurant,Sushi Restaurant
4,1400000US17031061800,1340,73.523,4.6,142,8.2769,3,161.896362,10.597015,2.238806,41.946435,-87.646811,9.0,Mexican Restaurant,Restaurant


Let's fill with '-' on the last 2 variables to indicate that there are no restaurants.

In [41]:
df_merged3['1st Most Common Venue'].replace(np.nan, '-', inplace=True)
df_merged3['2nd Most Common Venue'].replace(np.nan, '-', inplace=True)
df_merged3.head()

,GEO_ID,population,income,unemployment,latinos,area,crime count,density,latinos rate,crime rate,Latitude,Longitude,number restaurants,1st Most Common Venue,2nd Most Common Venue
0,1400000US17031051300,2826,178.750,1.3,185,32.2017,1,87.759342,6.546355,0.353857,41.935755,-87.676010,0.0,-,-
1,1400000US17031051400,1796,116.364,3.3,143,34.6283,0,51.865093,7.962138,0.000000,41.936528,-87.680827,1.0,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
2,1400000US17031060900,7114,59.942,1.9,1162,49.9067,4,142.545991,16.333989,0.562272,41.949074,-87.645666,2.0,Japanese Restaurant,Italian Restaurant
3,1400000US17031061500,1888,125.000,2.7,175,16.2913,1,115.890076,9.269068,0.529661,41.945462,-87.661520,4.0,American Restaurant,Sushi Restaurant
4,1400000US17031061800,1340,73.523,4.6,142,8.2769,3,161.896362,10.597015,2.238806,41.946435,-87.646811,9.0,Mexican Restaurant,Restaurant


Finally, we have our dataset ready for exploratory data analysis and modeling.

## Exploratory Data Analysis

In this section, let's provide some assumptions of where we should consider to open a new Peruvian restaurant by exploring some maps.

In [42]:
df_merged3.describe()

,population,income,unemployment,latinos,area,crime count,density,latinos rate,crime rate,Latitude,Longitude,number restaurants
count,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000
mean,3210.364078,77.789272,7.009223,589.660194,47.366407,32.271845,91.803376,19.204678,7.506204,41.888623,-87.661718,6.242718
std,2170.519603,41.788994,7.019649,795.098258,42.508244,107.679714,61.015943,22.007899,19.091686,0.034880,0.026724,7.645137
min,655.000000,10.455000,0.000000,0.000000,6.909400,0.000000,8.451170,0.000000,0.000000,41.825942,-87.710439,0.000000
25%,1808.500000,43.471250,2.625000,139.750000,23.448750,2.000000,49.565716,4.720659,0.659932,41.857379,-87.684217,1.000000
50%,2795.000000,76.266500,4.750000,290.000000,32.776750,6.000000,83.207751,8.429472,2.547896,41.893316,-87.660597,4.000000
75%,3824.000000,108.826750,9.075000,709.000000,56.706800,18.750000,110.978194,25.874582,6.275551,41.915656,-87.641163,8.750000
max,19889.000000,178.750000,44.000000,4738.000000,279.358600,1124.000000,397.296982,92.435257,157.615481,41.949074,-87.606843,51.000000


First of all, a map of the household income was made. The assumption is the following: the company should consider to open a new Peruvian restaurant in those census tracts with high income. This guarantees a high chance of visiting its restaurant and comssuming Peruvian cuisine. Census tracts in the center of CBD and north of the study area have high income.

In [43]:
# Reading Census tracts from GitHub
geopath = 'https://raw.githubusercontent.com/johans27/Coursera_Capstone/master/CtractsChicagoCBD.json'

# Alternative method to read census tracts from Github
# url = 'https://raw.githubusercontent.com/johans27/Coursera_Capstone/master'
# geopath = f'{url}/CtractsChicago.json'

# Cuts for the income (quantile)
threshold_scale = [10, 43, 76, 109, 179]

# Create a map with coordinates of the center of CBD in Chicago
chicago_map = folium.Map(location = [41.876378, -87.627946], zoom_start = 12)

chicago_map.choropleth(
    geo_data = geopath,
    data = df_merged3,
    columns = ['GEO_ID', 'income'],
    key_on = 'feature.properties.AFFGEOID',
    threshold_scale = threshold_scale,
    fill_color = 'Blues',
    fill_opacity = 0.7,
    line_opacity = 0.1,
    legend_name = '2018 Household Income of last 12 months (Thousands of dollars)'
)

# Display the map
chicago_map

Next, unemployment is an important variable since the company or contractor looks for areas where people is working and have a stable employment. So, they want to avoid areas with high rates of unemployment as they can be mainly seen in the west and south.

In [44]:
# Cuts for the unemployment rate (quantile)
threshold_scale = [0, 2.6, 4.8, 9, 45]

# Create a map with coordinates of the center of CBD in Chicago
chicago_map = folium.Map(location = [41.876378, -87.627946], zoom_start = 12)

chicago_map.choropleth(
    geo_data = geopath,
    data = df_merged3,
    columns = ['GEO_ID', 'unemployment'],
    key_on = 'feature.properties.AFFGEOID',
    threshold_scale = threshold_scale,
    fill_color = 'RdPu',
    fill_opacity = 0.7,
    line_opacity = 0.1,
    legend_name = '2018 Unemployment Rate (%)'
)

# Display the map
chicago_map

Then, populated areas might guarantee a high chance of visiting a commercial store as restaurants. So, companies are always looking for populated areas represented through the population density variable in this project. The north and center areas have high density.

In [45]:
# Cuts for the density (quantile)
threshold_scale = [8, 50, 83, 111, 398]

# Create a map with coordinates of the center of CBD in Chicago
chicago_map = folium.Map(location = [41.876378, -87.627946], zoom_start = 12)

chicago_map.choropleth(
    geo_data = geopath,
    data = df_merged3,
    columns = ['GEO_ID', 'density'],
    key_on = 'feature.properties.AFFGEOID',
    threshold_scale = threshold_scale,
    fill_color = 'Greys',
    fill_opacity = 0.7,
    line_opacity = 0.1,
    legend_name = '2018 Population Density'
)

# Display the map
chicago_map

The variable of latino population rate is not highly significant but it would be a plus if the new Peruvian restaurant is located in areas where latino population predominates. Here the northwest and soutwest have a high proportion of latinos.

In [46]:
# Cuts for the latino rate (quantile)
threshold_scale = [0, 4.7, 8.5, 26, 93]

# Create a map with coordinates of the center of CBD in Chicago
chicago_map = folium.Map(location = [41.876378, -87.627946], zoom_start = 12)

chicago_map.choropleth(
    geo_data = geopath,
    data = df_merged3,
    columns = ['GEO_ID', 'latinos rate'],
    key_on = 'feature.properties.AFFGEOID',
    threshold_scale = threshold_scale,
    fill_color = 'YlOrRd',
    fill_opacity = 0.7,
    line_opacity = 0.1,
    legend_name = '2018 Latino Population Rate (%)'
)

# Display the map
chicago_map

Since the Chicago Data Portal provides crimes of different types and where they happened, it was important to identify most common crimes as assaults, battery, burglary, robbery and thefts in restaurants. So, it tells the contractor to think 2 times if it would be a good idea to open a restaurant in an area with high crime rates in restaurants. Clearly from the map, the center of the study area is the most affected by those kind of crime.

In [47]:
# Cuts for the crime rate (manual)
threshold_scale = [0, 10, 50, 100, 158]

# Create a map with coordinates of the center of CBD in Chicago
chicago_map = folium.Map(location = [41.876378, -87.627946], zoom_start = 12)

chicago_map.choropleth(
    geo_data = geopath,
    data = df_merged3,
    columns = ['GEO_ID', 'crime rate'],
    key_on = 'feature.properties.AFFGEOID',
    threshold_scale = threshold_scale,
    fill_color = 'Reds',
    fill_opacity = 0.7,
    line_opacity = 0.1,
    legend_name = '2018-2019 Crimes by 1000 habitants'
)

# Display the map
chicago_map

Additionally, a high number of restaurants might reduce the chance of visiting the new restaurant. The idea is to avoid census tracts with high number of restaurants as it is observed in the north and center of the study area.

In [48]:
# Cuts for the number of restaurants (quantile)
threshold_scale = [0, 1, 4, 9, 52]

# Create a map with coordinates of the center of CBD in Chicago
chicago_map = folium.Map(location = [41.876378, -87.627946], zoom_start = 12)

chicago_map.choropleth(
    geo_data = geopath,
    data = df_merged3,
    columns = ['GEO_ID', 'number restaurants'],
    key_on = 'feature.properties.AFFGEOID',
    threshold_scale = threshold_scale,
    fill_color = 'PuRd',
    fill_opacity = 0.7,
    line_opacity = 0.1,
    legend_name = 'Number of restaurants'
)

# Display the map
chicago_map

Having explored these 6 variables we can create a first normalized score to get a first sense where to open a new Peruvian restaurant. Furthermore, we can create a second score by observing the top 2 most common restaurants that we can remember they come from the Foursquare analysis. Finally, sum up both scores and we can get a map of potential census tracts with high scores to open a new Peruvian restaurant.

## Modeling

In this section, let's create a score to show in a map the potential areas to open a new Peruvian restaurant. First of all, we need to follow a normalization process. Normalization is the process of transforming values of several variables into a similar range. There are several types of normalizations but this project considers scaling variable so the variable values range from 0 to 1. After normalizing variables we can sum up and obtain a first score for potential areas to open a new Peruvian restaurant.

In [49]:
# Let's explore our dataframe
df_merged3.head()

,GEO_ID,population,income,unemployment,latinos,area,crime count,density,latinos rate,crime rate,Latitude,Longitude,number restaurants,1st Most Common Venue,2nd Most Common Venue
0,1400000US17031051300,2826,178.750,1.3,185,32.2017,1,87.759342,6.546355,0.353857,41.935755,-87.676010,0.0,-,-
1,1400000US17031051400,1796,116.364,3.3,143,34.6283,0,51.865093,7.962138,0.000000,41.936528,-87.680827,1.0,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
2,1400000US17031060900,7114,59.942,1.9,1162,49.9067,4,142.545991,16.333989,0.562272,41.949074,-87.645666,2.0,Japanese Restaurant,Italian Restaurant
3,1400000US17031061500,1888,125.000,2.7,175,16.2913,1,115.890076,9.269068,0.529661,41.945462,-87.661520,4.0,American Restaurant,Sushi Restaurant
4,1400000US17031061800,1340,73.523,4.6,142,8.2769,3,161.896362,10.597015,2.238806,41.946435,-87.646811,9.0,Mexican Restaurant,Restaurant


In [50]:
# Let's create new variables for normalized variables which starts with 'N'
df_merged3['Nincome'] = df_merged3['income']/df_merged3['income'].max()
df_merged3['Nunemployment'] = df_merged3['unemployment']/df_merged3['unemployment'].max()
df_merged3['Ndensity'] = df_merged3['density']/df_merged3['density'].max()
df_merged3['Nlatinos'] = df_merged3['latinos rate']/df_merged3['latinos rate'].max()
df_merged3['Ncrime'] = df_merged3['crime rate']/df_merged3['crime rate'].max()
df_merged3['Nrestaurants'] = df_merged3['number restaurants']/df_merged3['number restaurants'].max()
df_merged3.describe()

,population,income,unemployment,latinos,area,crime count,density,latinos rate,crime rate,Latitude,Longitude,number restaurants,Nincome,Nunemployment,Ndensity,Nlatinos,Ncrime,Nrestaurants
count,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000
mean,3210.364078,77.789272,7.009223,589.660194,47.366407,32.271845,91.803376,19.204678,7.506204,41.888623,-87.661718,6.242718,0.435185,0.159301,0.231070,0.207764,0.047624,0.122406
std,2170.519603,41.788994,7.019649,795.098258,42.508244,107.679714,61.015943,22.007899,19.091686,0.034880,0.026724,7.645137,0.233785,0.159537,0.153578,0.238090,0.121128,0.149905
min,655.000000,10.455000,0.000000,0.000000,6.909400,0.000000,8.451170,0.000000,0.000000,41.825942,-87.710439,0.000000,0.058490,0.000000,0.021272,0.000000,0.000000,0.000000
25%,1808.500000,43.471250,2.625000,139.750000,23.448750,2.000000,49.565716,4.720659,0.659932,41.857379,-87.684217,1.000000,0.243196,0.059659,0.124757,0.051070,0.004187,0.019608
50%,2795.000000,76.266500,4.750000,290.000000,32.776750,6.000000,83.207751,8.429472,2.547896,41.893316,-87.660597,4.000000,0.426666,0.107955,0.209435,0.091193,0.016165,0.078431
75%,3824.000000,108.826750,9.075000,709.000000,56.706800,18.750000,110.978194,25.874582,6.275551,41.915656,-87.641163,8.750000,0.608821,0.206250,0.279333,0.279921,0.039816,0.171569
max,19889.000000,178.750000,44.000000,4738.000000,279.358600,1124.000000,397.296982,92.435257,157.615481,41.949074,-87.606843,51.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [51]:
# Building a weighted average for the first score based on the 6 variables of analysis
df_merged3['score1'] = ((5*df_merged3['Nincome']) - (3*df_merged3['Nunemployment']) + (5*df_merged3['Ndensity']) + (2*df_merged3['Nlatinos']) - (3*df_merged3['Ncrime']) - (2*df_merged3['Nrestaurants']))/20
df_merged3.describe()

,population,income,unemployment,latinos,area,crime count,density,latinos rate,crime rate,Latitude,Longitude,number restaurants,Nincome,Nunemployment,Ndensity,Nlatinos,Ncrime,Nrestaurants,score1
count,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000,206.000000
mean,3210.364078,77.789272,7.009223,589.660194,47.366407,32.271845,91.803376,19.204678,7.506204,41.888623,-87.661718,6.242718,0.435185,0.159301,0.231070,0.207764,0.047624,0.122406,0.144061
std,2170.519603,41.788994,7.019649,795.098258,42.508244,107.679714,61.015943,22.007899,19.091686,0.034880,0.026724,7.645137,0.233785,0.159537,0.153578,0.238090,0.121128,0.149905,0.086008
min,655.000000,10.455000,0.000000,0.000000,6.909400,0.000000,8.451170,0.000000,0.000000,41.825942,-87.710439,0.000000,0.058490,0.000000,0.021272,0.000000,0.000000,0.000000,-0.119837
25%,1808.500000,43.471250,2.625000,139.750000,23.448750,2.000000,49.565716,4.720659,0.659932,41.857379,-87.684217,1.000000,0.243196,0.059659,0.124757,0.051070,0.004187,0.019608,0.091453
50%,2795.000000,76.266500,4.750000,290.000000,32.776750,6.000000,83.207751,8.429472,2.547896,41.893316,-87.660597,4.000000,0.426666,0.107955,0.209435,0.091193,0.016165,0.078431,0.160106
75%,3824.000000,108.826750,9.075000,709.000000,56.706800,18.750000,110.978194,25.874582,6.275551,41.915656,-87.641163,8.750000,0.608821,0.206250,0.279333,0.279921,0.039816,0.171569,0.206691
max,19889.000000,178.750000,44.000000,4738.000000,279.358600,1124.000000,397.296982,92.435257,157.615481,41.949074,-87.606843,51.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.314055


Let's visualize this first score to get a sense where a contractor can consider to open a new Peruvian restaurant taking into consideration the 6 main variables based on social census data, crime and number of restaurants.

In [52]:
# Cuts for the first score (quantil)
threshold_scale = [-0.12, 0.091453, 0.160106, 0.206691, 0.32]

# Create a map with coordinates of the center of CBD in Chicago
chicago_map = folium.Map(location = [41.876378, -87.627946], zoom_start = 12)

chicago_map.choropleth(
    geo_data = geopath,
    data = df_merged3,
    columns = ['GEO_ID', 'score1'],
    key_on = 'feature.properties.AFFGEOID',
    threshold_scale = threshold_scale,
    fill_color = 'RdPu',
    fill_opacity = 0.7,
    line_opacity = 0.1,
    legend_name = 'Potential areas scores'
)

# Display the map
chicago_map

From the map, it is clearly seen that the north and close to the center of CBD in Chicago, are potential areas to start a business related to cuisine. Specifically, we are interested in opening a Peruvian restaurant. However, we should also consider if in these areas predominate latin american cuisine. If so, the likelihood of getting revenues is low because there will be more competitors with similar characteristics to the Peruvian cuisine. In that sense, let's assign a second score based on the 2 variables related to the top 2 most common restaurants. If a census tract has a Latin American restaurant (Argentinian, Brazilian, Caribbean, Cuban, Latin American, Mexican, Peruvian and South American) in one of 2 variables, its score will be 1, otherwise 3.

In [53]:
row_indexes = df_merged3[df_merged3['1st Most Common Venue'].str.contains('Argentinian|Brazilian|Caribbean|Cuban|Latin American|Mexican|Peruvian|South American') | df_merged3['2nd Most Common Venue'].str.contains('Argentinian|Brazilian|Caribbean|Cuban|Latin American|Mexican|Peruvian|South American')].index
df_merged3.loc[row_indexes,'score2'] = 1
df_merged3.head()

,GEO_ID,population,income,unemployment,latinos,area,crime count,density,latinos rate,crime rate,Latitude,Longitude,number restaurants,1st Most Common Venue,2nd Most Common Venue,Nincome,Nunemployment,Ndensity,Nlatinos,Ncrime,Nrestaurants,score1,score2
0,1400000US17031051300,2826,178.750,1.3,185,32.2017,1,87.759342,6.546355,0.353857,41.935755,-87.676010,0.0,-,-,1.000000,0.029545,0.220891,0.070821,0.002245,0.000000,0.307536,NaN
1,1400000US17031051400,1796,116.364,3.3,143,34.6283,0,51.865093,7.962138,0.000000,41.936528,-87.680827,1.0,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,0.650987,0.075000,0.130545,0.086137,0.000000,0.019608,0.190786,NaN
2,1400000US17031060900,7114,59.942,1.9,1162,49.9067,4,142.545991,16.333989,0.562272,41.949074,-87.645666,2.0,Japanese Restaurant,Italian Restaurant,0.335340,0.043182,0.358790,0.176707,0.003567,0.039216,0.180269,NaN
3,1400000US17031061500,1888,125.000,2.7,175,16.2913,1,115.890076,9.269068,0.529661,41.945462,-87.661520,4.0,American Restaurant,Sushi Restaurant,0.699301,0.061364,0.291696,0.100276,0.003360,0.078431,0.240225,NaN
4,1400000US17031061800,1340,73.523,4.6,142,8.2769,3,161.896362,10.597015,2.238806,41.946435,-87.646811,9.0,Mexican Restaurant,Restaurant,0.411317,0.104545,0.407495,0.114643,0.014204,0.176471,0.180708,1.0


In [54]:
df_merged3['score2'].replace(np.nan, 3, inplace=True)
# Sum up both scores
df_merged3['final_score'] = df_merged3['score1'] + df_merged3['score2']
df_merged3.head()

,GEO_ID,population,income,unemployment,latinos,area,crime count,density,latinos rate,crime rate,Latitude,Longitude,number restaurants,1st Most Common Venue,2nd Most Common Venue,Nincome,Nunemployment,Ndensity,Nlatinos,Ncrime,Nrestaurants,score1,score2,final_score
0,1400000US17031051300,2826,178.750,1.3,185,32.2017,1,87.759342,6.546355,0.353857,41.935755,-87.676010,0.0,-,-,1.000000,0.029545,0.220891,0.070821,0.002245,0.000000,0.307536,3.0,3.307536
1,1400000US17031051400,1796,116.364,3.3,143,34.6283,0,51.865093,7.962138,0.000000,41.936528,-87.680827,1.0,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,0.650987,0.075000,0.130545,0.086137,0.000000,0.019608,0.190786,3.0,3.190786
2,1400000US17031060900,7114,59.942,1.9,1162,49.9067,4,142.545991,16.333989,0.562272,41.949074,-87.645666,2.0,Japanese Restaurant,Italian Restaurant,0.335340,0.043182,0.358790,0.176707,0.003567,0.039216,0.180269,3.0,3.180269
3,1400000US17031061500,1888,125.000,2.7,175,16.2913,1,115.890076,9.269068,0.529661,41.945462,-87.661520,4.0,American Restaurant,Sushi Restaurant,0.699301,0.061364,0.291696,0.100276,0.003360,0.078431,0.240225,3.0,3.240225
4,1400000US17031061800,1340,73.523,4.6,142,8.2769,3,161.896362,10.597015,2.238806,41.946435,-87.646811,9.0,Mexican Restaurant,Restaurant,0.411317,0.104545,0.407495,0.114643,0.014204,0.176471,0.180708,1.0,1.180708


In [55]:
# Let's check the cuts of final score for mapping
df_merged3['final_score'].describe()

count    206.000000
mean       2.435323
std        0.938908
min        1.008886
25%        1.214778
50%        3.045956
75%        3.163673
max        3.314055
Name: final_score, dtype: float64

Let's see our map with potential areas!

In [56]:
# Cuts for the final score (quantile)
threshold_scale = [1, 1.214778, 3.045956, 3.163673, 3.4]

# Create a map with coordinates of the center of CBD in Chicago
chicago_map = folium.Map(location = [41.876378, -87.627946], zoom_start = 12)

chicago_map.choropleth(
    geo_data = geopath,
    data = df_merged3,
    columns = ['GEO_ID', 'final_score'],
    key_on = 'feature.properties.AFFGEOID',
    threshold_scale = threshold_scale,
    fill_color = 'RdPu',
    fill_opacity = 0.7,
    line_opacity = 0.1,
    legend_name = 'Potential areas scores to open a Peruvian restaurant'
)

# Display the map
chicago_map

The analysis suggests that any contractor or company interested in investing in a new Peruvian restaurant, should mainly locate in some areas of the north of the CBD of Chicago or close to it (scores above 3.2). However, areas at the south seem to be interesting as well given that there are no enough restaurants, crime rate is low and there are a lot of latino population. Finally, let's overlap Latin American restaurants on the map by creating a new variable called 'LA_venues'.

In [57]:
LA_venues = chicago_venues[chicago_venues['Restaurant Category'].str.contains('Argentinian|Brazilian|Caribbean|Cuban|Latin American|Mexican|Peruvian|South American')]
print(LA_venues.shape)
LA_venues.head()

(246, 7)


,GEO_ID,GEO_ID Latitude,GEO_ID Longitude,Restaurant,Restaurant Latitude,Restaurant Longitude,Restaurant Category
8,1400000US17031061800,41.946435,-87.646811,Taco & Burrito Palace,41.945653,-87.649464,Mexican Restaurant
10,1400000US17031061800,41.946435,-87.646811,Las Mañanitas,41.946222,-87.649428,Mexican Restaurant
15,1400000US17031061800,41.946435,-87.646811,Fajita Factory,41.944986,-87.649171,Mexican Restaurant
17,1400000US17031061901,41.942988,-87.643348,Esencia Urban Kitchen,41.943257,-87.644685,Mexican Restaurant
18,1400000US17031061901,41.942988,-87.643348,El Mariachi,41.943819,-87.645159,Mexican Restaurant


In [58]:
# Add Latin American restaurants to the map
for lat, lng, label in zip(LA_venues['Restaurant Latitude'], LA_venues['Restaurant Longitude'], LA_venues['Restaurant Category']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'grey',
        fill = True,
        fill_color = 'yellow',
        fill_opacity = 0.7,
        parse_html = False).add_to(chicago_map)  
    
chicago_map

By overlaping Latin American restaurants, we can see that a minimum number of them are located in the best areas to open a new Peruvian restaurant. These areas can be considered a great opportunity to invest in a Peruvian restaurant above all in the north where the income is high, unemployment rate is low, density population is somewhat high and crime rate is low.